In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 705 kB/s 


In [2]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

'2020 slides'
 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'API key'
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
'Review Group 9.gdoc'
 Xizhi_Wu_UnderGrad_Transcript.pdf
 教授资格证身份证.pdf


In [4]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [5]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_EU
 a0529-a0611_US		       dataframe_result_csv_UK
 a0608.csv		       Github
 a0609.csv		       heatmap529_UK.ipynb
 ccc_compiled.csv	       heatmap530.ipynb
 csv			       Heat_Map_Online_language.ipynb
 csv.zip		       __MACOSX
 dataframe_csv		       mapping_UK
 dataframe_csv_EU	       mapping_US
 dataframe_csv_UK	       non_violant1.csv
 dataframe_result_529_UK.csv   non_violant2.csv
 dataframe_result_530_UK.csv   UK
 dataframe_result_531_UK.csv   violant.csv
 dataframe_result_csv	      '新建 Microsoft PowerPoint 演示文稿.pptx'


In [6]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [7]:
ct=pd.read_csv('a0529-a0611_UK/a0604.csv',encoding='ISO-8859-1')

In [8]:
!ls

 a0529-a0611_UK		       dataframe_result_csv_EU
 a0529-a0611_US		       dataframe_result_csv_UK
 a0608.csv		       Github
 a0609.csv		       heatmap529_UK.ipynb
 ccc_compiled.csv	       heatmap530.ipynb
 csv			       Heat_Map_Online_language.ipynb
 csv.zip		       __MACOSX
 dataframe_csv		       mapping_UK
 dataframe_csv_EU	       mapping_US
 dataframe_csv_UK	       non_violant1.csv
 dataframe_result_529_UK.csv   non_violant2.csv
 dataframe_result_530_UK.csv   UK
 dataframe_result_531_UK.csv   violant.csv
 dataframe_result_csv	      '新建 Microsoft PowerPoint 演示文稿.pptx'


In [9]:
###
#ct=pd.read_csv('a0601.csv')

In [ ]:
ct

In [11]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [12]:
#ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [13]:
###
ct_ner = ct

In [14]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,{},{'America': 1},{}
1,{'Minneapolis': 1},{'Minneapolis': 1},{}
2,{},{},{}
3,{},{'Minneapolis': 2},{}
4,{},{'Minneapolis': 2},{}


In [15]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

In [16]:
ct_ner['NER-msg']

0                   []
1        [Minneapolis]
2                   []
3                   []
4                   []
             ...      
44686               []
44687               []
44688               []
44689               []
44690               []
Name: NER-msg, Length: 44691, dtype: object

In [17]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

In [18]:
ct_ner['NER']

0                         [America]
1        [Minneapolis, Minneapolis]
2                                []
3                     [Minneapolis]
4                     [Minneapolis]
                    ...            
44686                            []
44687                            []
44688                            []
44689                   [Hollywood]
44690                    [Illinois]
Name: NER, Length: 44691, dtype: object

In [19]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

In [20]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

In [21]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER
0,0,2054592981326565_2919503204835534,Facebook,2020-06-04 23:59:57,2021-04-24 15:46:24,youtube,Cornel West: The Future Of America Depends On ...,youtube.com,"Harvard University professor, author, and thin...",*** TAKE A GOOD LISTEN,1.0,https://www.youtube.com/watch?v=BkYJeMwlsto,https://www.facebook.com/groups/20545929813265...,2814,-30.000000,2.0,0,0,0,0,0,0,0,0,0,0,2,2,3,2,1,3,1,1,0,1,10176145,Bulgarians in USA United | ÐÐ±ÐµÐ´Ð¸Ð½ÐµÐ½Ð¸ ...,921thebeat,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,US,Tulsa's Party Station and home of Elvis Duran ...,False,2009-03-03 23:35:31,en,0,10176145|2919503204835534,[],[America],[],[america]
1,1,122310264455311_3256911610995145,Facebook,2020-06-04 23:59:57,2021-06-16 09:58:10,link,"T.I., Kevin Hart, Tiffany Haddish & More Atten...",wild104.iheart.com,The first of George Floyd's three public memor...,The first of George Floyd's three public memor...,1.0,http://ihe.art/yj2ad6P,https://www.facebook.com/122310264455311/posts...,140231,-10.000000,1.0,3,0,0,0,0,0,0,0,0,0,3,3,4,2,4,3,7,4,0,1,39988,WILD 104,wild104,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The Rio Grande Valley's Party Station for Hip...,True,2010-05-11 19:29:05,en,0,39988|3256911610995145,[Minneapolis],[Minneapolis],[],[minneapolis]
2,2,1250734815048111_3092238447564396,Facebook,2020-06-04 23:59:56,2021-08-16 19:32:24,link,Al Sharpton Attacks Trump At George Floydâs ...,chicagodailybrief.tumblr.com,Today at the funeral for George Floyd Reverend...,What a disgraceâ¦,1.0,https://chicagodailybrief.tumblr.com/post/6200...,https://www.facebook.com/groups/12507348150481...,28675,58.192308,1.0,44,120,588,2,9,17,60,673,0,0,5,2,4,3,2,5,2,3,0,3,7838630,FOX NEWS with Tucker Carlson,wild104,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,The Rio Grande Valley's Party Station for Hip...,False,2010-05-11 19:29:05,en,0,7838630|3092238447564396,[],[],[],[]
3,3,217290511618934_3551441844870434,Facebook,2020-06-04 23:59:56,2021-10-15 05:19:58,link,Reclaim the Block: fund our broader movement,docs.google.com,"To all our supporters, We are so grateful for ...",>READ ALL OF THIS BEFORE SEARCHING< Me and my ...,5.0,https://docs.google.com/document/d/1yLWGTQIe39...,https://www.facebook.com/217290511618934/posts...,37773,1.631579,1.0,23,0,1,7,0,0,0,0,0,0,8,0,1,3,0,0,0,0,0,0,9119246,Brad Kavanagh,officialbradkavanagh,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,GB,Music maker and one quarter of the band FLYNT.,False,2011-04-13 22:38:32,en,0,9119246|3551441844870434,[],[Minneapolis],[],[minneapolis]
4,4,79146833756_10157710305038757,Facebook,2020-06-04 23:59:54,2021-08-31 17:54:38,native_video,Reclaim the Block: fund our broader movement,docs.google.com,"To all our supporters, We are so grateful for ...",Protesters peaceully march near Water and Broa...,1.0,https://www.facebook.com/tmj4/videos/186296016...,https://www.facebook.com/79146833756/posts/101...,348655,3.456026,2.0,357,47,515,118,2,7,4,11,0,19,162,38,27,46,11,11,6,6,0,4,25507,TMJ4 News,tmj4,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,TMJ4 News brings you southeast Wisconsin break...,True,2009-05-13 13:33:43,en,0,25507|10157710305038757,[],[Minneapoli

In [22]:
ct_ner['NER-1']=ct_ner['NER']

In [23]:
ct_ner['NER-1'][1]

['minneapolis']

In [24]:
!pip install -q geopy
!pip install -q pyproj

# basic utilities
import json
import requests
import pandas as pd
import pprint
import time

# reverse geocoding utilities
from geopy.geocoders import Nominatim

# geography projection utilities
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
from functools import partial

     |████████████████████████████████| 6.3 MB 11.0 MB/s 


In [25]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
#from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [26]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,2054592981326565_2919503204835534,Facebook,2020-06-04 23:59:57,2021-04-24 15:46:24,youtube,Cornel West: The Future Of America Depends On ...,youtube.com,"Harvard University professor, author, and thin...",*** TAKE A GOOD LISTEN,1.0,https://www.youtube.com/watch?v=BkYJeMwlsto,https://www.facebook.com/groups/20545929813265...,2814,-30.000000,2.0,0,0,0,0,0,0,0,0,0,0,2,2,3,2,1,3,1,1,0,1,10176145,Bulgarians in USA United | ÐÐ±ÐµÐ´Ð¸Ð½ÐµÐ½Ð¸ ...,921thebeat,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,US,Tulsa's Party Station and home of Elvis Duran ...,False,2009-03-03 23:35:31,en,0,10176145|2919503204835534,[],[America],[],[america],[america]
1,1,122310264455311_3256911610995145,Facebook,2020-06-04 23:59:57,2021-06-16 09:58:10,link,"T.I., Kevin Hart, Tiffany Haddish & More Atten...",wild104.iheart.com,The first of George Floyd's three public memor...,The first of George Floyd's three public memor...,1.0,http://ihe.art/yj2ad6P,https://www.facebook.com/122310264455311/posts...,140231,-10.000000,1.0,3,0,0,0,0,0,0,0,0,0,3,3,4,2,4,3,7,4,0,1,39988,WILD 104,wild104,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The Rio Grande Valley's Party Station for Hip...,True,2010-05-11 19:29:05,en,0,39988|3256911610995145,[Minneapolis],[Minneapolis],[],[minneapolis],[minneapolis]
2,2,1250734815048111_3092238447564396,Facebook,2020-06-04 23:59:56,2021-08-16 19:32:24,link,Al Sharpton Attacks Trump At George Floydâs ...,chicagodailybrief.tumblr.com,Today at the funeral for George Floyd Reverend...,What a disgraceâ¦,1.0,https://chicagodailybrief.tumblr.com/post/6200...,https://www.facebook.com/groups/12507348150481...,28675,58.192308,1.0,44,120,588,2,9,17,60,673,0,0,5,2,4,3,2,5,2,3,0,3,7838630,FOX NEWS with Tucker Carlson,wild104,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,The Rio Grande Valley's Party Station for Hip...,False,2010-05-11 19:29:05,en,0,7838630|3092238447564396,[],[],[],[],[]
3,3,217290511618934_3551441844870434,Facebook,2020-06-04 23:59:56,2021-10-15 05:19:58,link,Reclaim the Block: fund our broader movement,docs.google.com,"To all our supporters, We are so grateful for ...",>READ ALL OF THIS BEFORE SEARCHING< Me and my ...,5.0,https://docs.google.com/document/d/1yLWGTQIe39...,https://www.facebook.com/217290511618934/posts...,37773,1.631579,1.0,23,0,1,7,0,0,0,0,0,0,8,0,1,3,0,0,0,0,0,0,9119246,Brad Kavanagh,officialbradkavanagh,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,GB,Music maker and one quarter of the band FLYNT.,False,2011-04-13 22:38:32,en,0,9119246|3551441844870434,[],[Minneapolis],[],[minneapolis],[minneapolis]
4,4,79146833756_10157710305038757,Facebook,2020-06-04 23:59:54,2021-08-31 17:54:38,native_video,Reclaim the Block: fund our broader movement,docs.google.com,"To all our supporters, We are so grateful for ...",Protesters peaceully march near Water and Broa...,1.0,https://www.facebook.com/tmj4/videos/186296016...,https://www.facebook.com/79146833756/posts/101...,348655,3.456026,2.0,357,47,515,118,2,7,4,11,0,19,162,38,27,46,11,11,6,6,0,4,25507,TMJ4 News,tmj4,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,TMJ4 News brings you southeast Wisconsin break...,True,2009-05-13 13:33

In [27]:
%cd mapping_UK

/content/gdrive/My Drive/2021-online-language-offline-impact-main/mapping_UK


In [28]:
!ls

dataframe0529_post.csv	    dataframe_mapping_0603.csv
dataframe0530_post.csv	    dataframe_mapping_0604.csv
dataframe0531_post.csv	    dataframe_mapping_0605.csv
dataframe0601_post.csv	    dataframe_mapping_0606.csv
dataframe0602_post.csv	    dataframe_mapping_0607.csv
dataframe_mapping_0529.csv  dataframe_mapping_0608.csv
dataframe_mapping_0530.csv  dataframe_mapping_0609.csv
dataframe_mapping_0531.csv  dataframe_mapping_0610.csv
dataframe_mapping_0601.csv  dataframe_mapping_0611.csv
dataframe_mapping_0602.csv


In [29]:
ct_mapping=pd.read_csv('dataframe_mapping_0604.csv',encoding='ISO-8859-1')

In [30]:
len(ct_mapping['City'])

4823

In [31]:
len(ct_ner['NER-1'])

44691

In [32]:
for k in range(len(ct_ner['NER-1'])):
  list=[]
  for i in ct_ner['NER-1'][k]:
    for j in range(len(ct_mapping['City'])):
      if(i==ct_mapping['City'][j]):
        list.append(ct_mapping['Mapping'][j])
  ct_ner['NER-1'][k]=list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [33]:
ct_ner

,Unnamed: 0,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title,NER,NER-1
0,0,2054592981326565_2919503204835534,Facebook,2020-06-04 23:59:57,2021-04-24 15:46:24,youtube,Cornel West: The Future Of America Depends On ...,youtube.com,"Harvard University professor, author, and thin...",*** TAKE A GOOD LISTEN,1.0,https://www.youtube.com/watch?v=BkYJeMwlsto,https://www.facebook.com/groups/20545929813265...,2814,-30.000000,2.0,0,0,0,0,0,0,0,0,0,0,2,2,3,2,1,3,1,1,0,1,10176145,Bulgarians in USA United | ÐÐ±ÐµÐ´Ð¸Ð½ÐµÐ½Ð¸ ...,921thebeat,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_group,US,Tulsa's Party Station and home of Elvis Duran ...,False,2009-03-03 23:35:31,en,0,10176145|2919503204835534,[],[America],[],[america],"[America, Horst aan de Maas, Limburg, Nederland]"
1,1,122310264455311_3256911610995145,Facebook,2020-06-04 23:59:57,2021-06-16 09:58:10,link,"T.I., Kevin Hart, Tiffany Haddish & More Atten...",wild104.iheart.com,The first of George Floyd's three public memor...,The first of George Floyd's three public memor...,1.0,http://ihe.art/yj2ad6P,https://www.facebook.com/122310264455311/posts...,140231,-10.000000,1.0,3,0,0,0,0,0,0,0,0,0,3,3,4,2,4,3,7,4,0,1,39988,WILD 104,wild104,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,The Rio Grande Valley's Party Station for Hip...,True,2010-05-11 19:29:05,en,0,39988|3256911610995145,[Minneapolis],[Minneapolis],[],[minneapolis],"[Minneapolis, Hennepin County, Minnesota, 5541..."
2,2,1250734815048111_3092238447564396,Facebook,2020-06-04 23:59:56,2021-08-16 19:32:24,link,Al Sharpton Attacks Trump At George Floydâs ...,chicagodailybrief.tumblr.com,Today at the funeral for George Floyd Reverend...,What a disgraceâ¦,1.0,https://chicagodailybrief.tumblr.com/post/6200...,https://www.facebook.com/groups/12507348150481...,28675,58.192308,1.0,44,120,588,2,9,17,60,673,0,0,5,2,4,3,2,5,2,3,0,3,7838630,FOX NEWS with Tucker Carlson,wild104,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_group,US,The Rio Grande Valley's Party Station for Hip...,False,2010-05-11 19:29:05,en,0,7838630|3092238447564396,[],[],[],[],[]
3,3,217290511618934_3551441844870434,Facebook,2020-06-04 23:59:56,2021-10-15 05:19:58,link,Reclaim the Block: fund our broader movement,docs.google.com,"To all our supporters, We are so grateful for ...",>READ ALL OF THIS BEFORE SEARCHING< Me and my ...,5.0,https://docs.google.com/document/d/1yLWGTQIe39...,https://www.facebook.com/217290511618934/posts...,37773,1.631579,1.0,23,0,1,7,0,0,0,0,0,0,8,0,1,3,0,0,0,0,0,0,9119246,Brad Kavanagh,officialbradkavanagh,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,GB,Music maker and one quarter of the band FLYNT.,False,2011-04-13 22:38:32,en,0,9119246|3551441844870434,[],[Minneapolis],[],[minneapolis],"[Minneapolis, Hennepin County, Minnesota, 5541..."
4,4,79146833756_10157710305038757,Facebook,2020-06-04 23:59:54,2021-08-31 17:54:38,native_video,Reclaim the Block: fund our broader movement,docs.google.com,"To all our supporters, We are so grateful for ...",Protesters peaceully march near Water and Broa...,1.0,https://www.facebook.com/tmj4/videos/186296016...,https://www.facebook.com/79146833756/posts/101...,348655,3.456026,2.0,357,47,515,118,2,7,4,11,0,19,162,38,27,46,11,11,6,6,0,4,25507,TMJ4 News,tmj4,https://scontent-sea1

In [34]:
ct_ner['NER-1'][12]

[]

In [35]:
ct_ner.to_csv('dataframe0604_post.csv')